<img src="https://geodecode.com.br/wp-content/uploads/2021/12/brasildatacube.png" align="left" width="74"/>
<img src="https://brazil-data-cube.github.io/_images/logo-odc-bdc.png" align="right" width="74" />

<h1 style="color:#336699; text-align: center" >Annual Metting Use Case</h1>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    <sup>
        <a href="https://orcid.org/0000-0002-9916-1167">
            <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i>
        </a>
    </sup>
    <br/>
    Brazil Data Cube (BDC), National Institute for Space Research (INPE)
    <br/>
    <br/>
    <br/>
    <div style="width: 98%;">
        <div style="text-align: justify; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 10px;">
            <b>Explantion here....:</b>
            <div>
                Write write write write writewrite writewrite write. <a href="http://www.brazildatacube.org/" target="_blank">Site site site site site</a>, <i> Outras infos</i> .......
            </div>
        </div>
    </div>
</div>

In [1]:
library(sf)
library(rstac)
library(dplyr)

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 8.2.1; sf_use_s2() is TRUE


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Set constants
<hr style="border:1px solid #0077b9;">

In [4]:
HARMONIZE_STAC_URL <- 'https://brazildatacube.dpi.inpe.br/harmonize/dev/stac/v1/'


START_DATE <- "2020-01-01"
END_DATE <- "2020-12-31"


FINAL_CSV_NAME <- "denguecases_months_2020.csv"

# List Harmonize STAC collections (using RSTAC)
<hr style="border:1px solid #0077b9;">

In [9]:
all_collections <- stac(HARMONIZE_STAC_URL) %>%
  collections() %>%
  get_request()

for (collection in all_collections$collections){
  print(collection$id)
}

[1] "dengue_mortality_rate_mun_year_north-1"
[1] "max_temper_mean_ne_mun_month-1"
[1] "total_prec_max_no_mun_epiweek-1"
[1] "Phantom3Advanced_FlightHeight120m_Mosaic-1"
[1] "Phantom3Advanced_FlightHeight120m-1"
[1] "chagas_disease_cases_mun_week_north-1"
[1] "max_temper_mean_no_mun_month-1"
[1] "max_temper_max_no_mun_month-1"
[1] "max_temper_max_no_mun_epiweek_ras-1"
[1] "chagas_disease_cases_mun_month_north-1"
[1] "max_temper_max_ne_mun_epiweek-1"
[1] "dengue_cases_mun_year_north-1"
[1] "chikungunya_cases_mun_year_north-1"
[1] "zika_cases_mun_month_north-1"
[1] "dengue_cases_mun_month_north-1"
[1] "chikungunya_cases_mun_month_north-1"
[1] "dengue_cases_mun_week_north-1"
[1] "dengue_mortality_rate_mun_month_north-1"
[1] "max_temper_mean_no_mun_epiweek-1"
[1] "zika_cases_mun_year_north-1"
[1] "chagas_disease_cases_mun_year_north-1"
[1] "zika_cases_mun_week_north-1"
[1] "total_prec_max_no_mun_month-1"
[1] "dengue_mortality_rate_mun_week_north-1"
[1] "chikungunya_cases_mun_week_north-1"
[

# Obtain monthly climate data for each municipality
<hr style="border:1px solid #0077b9;">

### Set collection id (temperature, precipitation, dengue, zika, etc)

In [10]:
# COLLECTION_ID <- "max_temper_mean_no_mun_month-1"
# COLLECTION_ID <- "total_prec_max_no_mun_month-1"
# COLLECTION_ID <- "total_prec_max_no_mun_epiweek-1"
# COLLECTION_ID <- "max_temper_mean_no_mun_epiweek-1"
COLLECTION_ID <- "dengue_cases_mun_month_north-1"
# COLLECTION_ID <- "dengue_mortality_rate_mun_month_north-1"
# COLLECTION_ID <- "dengue_cases_mun_week_north-1"
# [1] "total_prec_max_no_mun_month-1"
# [1] "total_prec_mean_no_mun_month-1"
# [1] "total_prec_mean_ne_mun_month-1"
# [1] "total_prec_max_ne_mun_month-1"


### Search climate data on Harmonize STAC (Using RSTAC)

In [11]:
stac_search <- stac(HARMONIZE_STAC_URL) %>%
  stac_search(
    collections = COLLECTION_ID,
    datetime = paste(START_DATE, END_DATE, sep = "/"),
    limit = 200
  ) %>%
  get_request()

stac_search$context

$matched
[1] 12

$returned
[1] 12

### For each feature, retrieve geojson as dataframe

In [13]:
feature_to_geojson_csv <- function(feature) {
  datetime <- feature$properties$datetime
  geojson_href <- feature$assets$geojson$href
  print(geojson_href)

  as.data.frame(st_read(geojson_href)) %>%
    mutate(date = format(as.Date(datetime), "%Y-%m-%d"))
}

dfs <- lapply(X = stac_search$features, FUN = feature_to_geojson_csv)
dfs[1:2]

[1] "https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/month/municipality/items/2020-12-01/north/dengue_cases_mun_month_north_20201201.geojson"
Reading layer `dengue_cases_mun_month_north_20201201' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/month/municipality/items/2020-12-01/north/dengue_cases_mun_month_north_20201201.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
[1] "https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/month/municipality/items/2020-11-01/north/dengue_cases_mun_month_north_20201101.geojson"
Reading layer `dengue_cases_mun_month_north_20201101' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/month/municipality/ite

cod,date,name,agg,agg_time,value,geometry
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<POLYGON [°]>
150010,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-48.91969 -1.9472...
150080,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-48.39812 -1.3927...
150110,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-49.94238 -2.7575...
150120,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-49.55923 -2.9383...
150130,2020-12-01,indi_0015,mun_res,month,0.79,POLYGON ((-48.66029 -1.6923...
150140,2020-12-01,indi_0015,mun_res,month,1.47,POLYGON ((-48.52311 -1.3820...
150150,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-48.28103 -1.3827...
150178,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-49.23882 -4.0763...
150210,2020-12-01,indi_0015,mun_res,month,0.0,POLYGON ((-49.26145 -2.0259...


### Rbind lists of dataframes in one big dataframe

##### If Health Data:

In [14]:
DATA_TYPE <- "dengue"
# DATA_TYPE <- "zika"


final_df <- do.call(rbind, dfs) %>%
  dplyr::select(cod, date, value) %>%
  rename(cd_mun = cod, !!DATA_TYPE := value)

dim(final_df)
head(final_df)

[1] 252   3

,cd_mun,date,dengue
,<chr>,<chr>,<chr>
1,150010,2020-12-01,0.0
2,150080,2020-12-01,0.0
3,150110,2020-12-01,0.0
4,150120,2020-12-01,0.0
5,150130,2020-12-01,0.79
6,150140,2020-12-01,1.47


##### If Climate Data:

In [13]:
DATA_TYPE <- "precipitation"
# DATA_TYPE <- "temperature"


final_df <- do.call(rbind, dfs) %>%
  dplyr::select(CD_MUN, NM_MUN, value, date) %>%
  rename(cd_mun = CD_MUN, nm_mun = NM_MUN, !!DATA_TYPE := value)

dim(final_df)
head(final_df)

ERROR: [1m[33mError[39m in `dplyr::select()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `CD_MUN` doesn't exist.


### Save data

In [12]:
write.csv(final_df, FINAL_CSV_NAME, row.names = FALSE)